# Logistic Regression Solver

In [ ]:
# Implemented in Julia 1.0
# Vivak Patel

#Import Data
using RDatasets
UCBAdmit = RDatasets.dataset("datasets","UCBAdmissions")

#Generate Observed Variables
Y = map(y -> y == "Admitted" ? 1 : 0, UCBAdmit[1])


┌ Info: Precompiling RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1186
